In [1]:
import pandas as pd
import re
import pymysql
from sqlalchemy import create_engine
import getpass

In [2]:
total_accidents_2018 = pd.read_excel('./files_for_lab/2018_Accidentalidad.xlsx')
total_accidents_2018

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE 15 A 17 AÑOS
1,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,NO ASIGNADO,TESTIGO,HOMBRE,IL,DE 30 A 34 ANOS
2,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
3,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,NO ASIGNADO,TESTIGO,HOMBRE,IL,DE 21 A 24 AÑOS
4,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,NO ASIGNADO,TESTIGO,MUJER,IL,DE 40 A 44 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30117,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA ...,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,VIAJERO,HOMBRE,IL,DE 30 A 34 ANOS
30118,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA ...,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,AUTO-TAXI,CONDUCTOR,HOMBRE,IL,DE 40 A 44 AÑOS
30119,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,MUJER,IL,DE 21 A 24 AÑOS
30120,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS


In [3]:
total_accidents_2018['TIPO PERSONA'].value_counts()

CONDUCTOR                                   19333
VIAJERO                                      5365
TESTIGO                                      3659
PEATON                                       1765
Name: TIPO PERSONA, dtype: int64

In [4]:
total_accidents_2018['LESIVIDAD'].value_counts()

IL                                          15574
HL                                          12497
NO ASIGNADA                                  1118
HG                                            896
MT                                             37
Name: LESIVIDAD, dtype: int64

In [5]:
total_accidents_2018.columns

Index(['FECHA', 'RANGO HORARIO', 'DIA SEMANA', 'DISTRITO', 'LUGAR ACCIDENTE',
       'Nº', 'Nº PARTE', 'CPFA Granizo', 'CPFA Hielo', 'CPFA Lluvia',
       'CPFA Niebla', 'CPFA Seco', 'CPFA Nieve', 'CPSV Mojada', 'CPSV Aceite',
       'CPSV Barro', 'CPSV Grava Suelta', 'CPSV Hielo', 'CPSV Seca Y Limpia',
       'Nº VICTIMAS', 'TIPO ACCIDENTE', 'Tipo Vehiculo', 'TIPO PERSONA',
       'SEXO', 'LESIVIDAD', 'Tramo Edad'],
      dtype='object')

In [6]:
total_accidents_2018.isna().sum()

FECHA                 0
RANGO HORARIO         0
DIA SEMANA            0
DISTRITO              0
LUGAR ACCIDENTE       0
Nº                    0
Nº PARTE              0
CPFA Granizo          0
CPFA Hielo            0
CPFA Lluvia           0
CPFA Niebla           0
CPFA Seco             0
CPFA Nieve            0
CPSV Mojada           0
CPSV Aceite           0
CPSV Barro            0
CPSV Grava Suelta     0
CPSV Hielo            0
CPSV Seca Y Limpia    0
Nº VICTIMAS           0
TIPO ACCIDENTE        0
Tipo Vehiculo         0
TIPO PERSONA          0
SEXO                  0
LESIVIDAD             0
Tramo Edad            0
dtype: int64

In [7]:
total_accidents_2018['DISTRITO'].unique()

array(['USERA                         ', 'HORTALEZA                     ',
       'SAN BLAS                      ', 'CIUDAD LINEAL                 ',
       'FUENCARRAL-EL PARDO           ', 'MORATALAZ                     ',
       'ARGANZUELA                    ', 'CENTRO                        ',
       'SALAMANCA                     ', 'CHAMARTIN                     ',
       'CARABANCHEL                   ', 'RETIRO                        ',
       'TETUAN                        ', 'MONCLOA-ARAVACA               ',
       'CHAMBERI                      ', 'VICALVARO                     ',
       'VILLAVERDE                    ', 'VILLA DE VALLECAS             ',
       'PUENTE DE VALLECAS            ', 'LATINA                        ',
       'BARAJAS                       '], dtype=object)

In [8]:
for col in total_accidents_2018.columns:
    print(total_accidents_2018[col].unique())

['2018-01-01T00:00:00.000000000' '2018-01-02T00:00:00.000000000'
 '2018-01-03T00:00:00.000000000' '2018-01-04T00:00:00.000000000'
 '2018-01-05T00:00:00.000000000' '2018-01-06T00:00:00.000000000'
 '2018-01-07T00:00:00.000000000' '2018-01-08T00:00:00.000000000'
 '2018-01-09T00:00:00.000000000' '2018-01-10T00:00:00.000000000'
 '2018-01-11T00:00:00.000000000' '2018-01-12T00:00:00.000000000'
 '2018-01-13T00:00:00.000000000' '2018-01-14T00:00:00.000000000'
 '2018-01-15T00:00:00.000000000' '2018-01-16T00:00:00.000000000'
 '2018-01-17T00:00:00.000000000' '2018-01-18T00:00:00.000000000'
 '2018-01-19T00:00:00.000000000' '2018-01-20T00:00:00.000000000'
 '2018-01-21T00:00:00.000000000' '2018-01-22T00:00:00.000000000'
 '2018-01-23T00:00:00.000000000' '2018-01-24T00:00:00.000000000'
 '2018-01-25T00:00:00.000000000' '2018-01-26T00:00:00.000000000'
 '2018-01-27T00:00:00.000000000' '2018-01-28T00:00:00.000000000'
 '2018-01-29T00:00:00.000000000' '2018-01-30T00:00:00.000000000'
 '2018-01-31T00:00:00.000

In [9]:
total_accidents_2017 = pd.read_excel('./files_for_lab/2017_Accidentalidad.xlsx')
total_accidents_2017

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM ...,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 25 A 29 AÑOS
1,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM ...,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,MUJER,HL,DE 30 A 34 ANOS
2,2017-01-01,DE 2:00 A 2:59,DOMINGO,VILLAVERDE,CALLE DE SANTA JOAQUINA DE VEDRUNA NUM ...,3,2017/1,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,HOMBRE,HG,DE 21 A 24 AÑOS
3,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM. ...,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,MUJER,HL,DE 45 A 49 AÑOS
4,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM. ...,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,MUJER,IL,DE 10 A 14 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29790,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA ...,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 65 A 69 AÑOS
29791,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA ...,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,AUTOBUS-AUTOCAR,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
29792,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM ...,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS
29793,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM ...,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,MUJER,IL,DE 45 A 49 AÑOS


In [10]:
total_accidents_2016 = pd.read_excel('./files_for_lab/2016_Accidentalidad.xlsx')
total_accidents_2016

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 6 A 9 AÑOS
1,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 40 A 44 AÑOS
2,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS
3,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,MUJER,HL,DE 35 A 39 AÑOS
4,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,TESTIGO,MUJER,IL,DE 40 A 44 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29196,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM ...,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,NO ASIGNADO,NO ASIGNADA,DESCONOCIDA
29197,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM ...,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,MUJER,HL,DE 6 A 9 AÑOS
29198,2016-12-31,DE 20:00 A 20:59,SABADO,CARABANCHEL,CALLE DE EUGENIA DE MONTIJO NUM ...,83,2017/14,NO,NO,NO,...,NO,NO,SI,1,CAÍDA MOTOCICLETA,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS
29199,2016-12-31,DE 23:00 A 23:59,SABADO,MONCLOA-ARAVACA,PASEO DEL PINTOR ROSALES NUM ...,44,2017/35,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS


In [11]:
def clean_file(row):
    #Changing headers names
    cols1 = []
    for column in row.columns:
        new_column_name = column.replace(' ', '_')
        new_column_name = new_column_name.replace('º', 'ro')
        cols1.append(new_column_name.lower())
    row.columns = cols1
    # I'm going to clean this columns because all values have spaces at the start and at the end
    list_to_clean = ['distrito', 'lugar_accidente', 'tipo_accidente', 'tipo_vehiculo', 'tipo_persona', 'lesividad']
    for column in list_to_clean:
        for i in range(len(row['lesividad'])):
            row[column][i] = str(row[column][i])
            row[column][i] = row[column][i].strip()
    row = row.reset_index(drop = True)
    # I don't need witness("TESTIGO") because i want statistics about people who was involved into the accident
    row = row.drop(row[row['tipo_persona']=='TESTIGO'].index)
    row = row.reset_index(drop = True)
    return row

In [12]:
total_accidents_2018['DIA SEMANA'].value_counts()

VIERNES      5308
MIERCOLES    4660
MARTES       4570
JUEVES       4480
LUNES        4430
SABADO       3463
DOMINGO      3211
Name: DIA SEMANA, dtype: int64

In [13]:
total_accidents_2016['TIPO PERSONA'].value_counts()

CONDUCTOR                                   18575
VIAJERO                                      5189
TESTIGO                                      3724
PEATON                                       1713
Name: TIPO PERSONA, dtype: int64

In [14]:
clean_2016 = clean_file(total_accidents_2016)
clean_2016

<ipython-input-11-dc4b31b9055a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = str(row[column][i])
<ipython-input-11-dc4b31b9055a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = row[column][i].strip()


,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 6 A 9 AÑOS
1,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 40 A 44 AÑOS
2,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS
3,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,MUJER,HL,DE 35 A 39 AÑOS
4,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 30 A 34 ANOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25472,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,NO ASIGNADO,NO ASIGNADA,DESCONOCIDA
25473,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,MUJER,HL,DE 6 A 9 AÑOS
25474,2016-12-31,DE 20:00 A 20:59,SABADO,CARABANCHEL,CALLE DE EUGENIA DE MONTIJO NUM,83,2017/14,NO,NO,NO,...,NO,NO,SI,1,CAÍDA MOTOCICLETA,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS
25475,2016-12-31,DE 23:00 A 23:59,SABADO,MONCLOA-ARAVACA,PASEO DEL PINTOR ROSALES NUM,44,2017/35,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS


In [15]:
total_accidents_2017['TIPO PERSONA'].value_counts()

CONDUCTOR                                   19105
VIAJERO                                      5304
TESTIGO                                      3699
PEATON                                       1687
Name: TIPO PERSONA, dtype: int64

In [16]:
clean_2017 = clean_file(total_accidents_2017)
clean_2017

<ipython-input-11-dc4b31b9055a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = str(row[column][i])
<ipython-input-11-dc4b31b9055a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = row[column][i].strip()


,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 25 A 29 AÑOS
1,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,MUJER,HL,DE 30 A 34 ANOS
2,2017-01-01,DE 2:00 A 2:59,DOMINGO,VILLAVERDE,CALLE DE SANTA JOAQUINA DE VEDRUNA NUM,3,2017/1,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,HOMBRE,HG,DE 21 A 24 AÑOS
3,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM.,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,MUJER,HL,DE 45 A 49 AÑOS
4,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM.,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,MUJER,IL,DE 10 A 14 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26091,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 65 A 69 AÑOS
26092,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,AUTOBUS-AUTOCAR,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
26093,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS
26094,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,MUJER,IL,DE 45 A 49 AÑOS


In [17]:
clean_2018 = clean_file(total_accidents_2018)
clean_2018

<ipython-input-11-dc4b31b9055a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = str(row[column][i])
<ipython-input-11-dc4b31b9055a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = row[column][i].strip()


,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE 15 A 17 AÑOS
1,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
2,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS
3,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,HOMBRE,HG,DE 21 A 24 AÑOS
4,2018-01-01,DE 1:00 A 1:59,LUNES,SAN BLAS,AVENIDA DE CANILLEJAS A VICALVARO - CALLE DE A...,0,2018/9,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 25 A 29 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26458,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,VIAJERO,HOMBRE,IL,DE 30 A 34 ANOS
26459,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,AUTO-TAXI,CONDUCTOR,HOMBRE,IL,DE 40 A 44 AÑOS
26460,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,MUJER,IL,DE 21 A 24 AÑOS
26461,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS


In [25]:
total_accidents =pd.concat([clean_2016, clean_2017, clean_2018], axis =0)

In [26]:
total_accidents = total_accidents.reset_index(drop = True)

In [27]:
total_accidents

,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 6 A 9 AÑOS
1,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 40 A 44 AÑOS
2,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS
3,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,MUJER,HL,DE 35 A 39 AÑOS
4,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 30 A 34 ANOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,VIAJERO,HOMBRE,IL,DE 30 A 34 ANOS
78032,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,AUTO-TAXI,CONDUCTOR,HOMBRE,IL,DE 40 A 44 AÑOS
78033,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,MUJER,IL,DE 21 A 24 AÑOS
78034,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS


In [28]:
def translate(row):
    row = row.rename(columns={"fecha": "date", "rango_horario": "hour", "dia_semana": "week_day", "distrito": "district", "lugar_accidente": "street", "nro": "number", "nro_parte": "id_accident", "cpfa_granizo": "ac_hail", "cpfa_hielo": "ac_ice", "cpfa_lluvia": "ac_rain", "cpfa_niebla": "ac_fog", "cpfa_seco": "ac_dry", "cpfa_nieve": "ac_snow", "cpsv_mojada": "wet", "cpsv_aceite": "oil", "cpsv_barro": "mud", "cpsv_grava_suelta": "gravel", "cpsv_hielo": "ice", "cpsv_seca_y_limpia": "dry_and_clean", "nro_victimas": "number_of_victims", "tipo_accidente": "type_of_accident", "tipo_vehiculo": "vehicle", "tipo_persona": "type_of_person", "sexo": "gender", "lesividad": "gravity", "tramo_edad": "age"})
    pattern = 'DE '
    pattern1 = ' A '
    for i in range(len(row['hour'])):
        row['hour'][i] = re.sub(pattern, '',row['hour'][i])
        row['hour'][i] = re.sub(pattern1, '-',row['hour'][i])
    #row['hour'] = row['hour'].replace("DE","BETWEEN")
    #row['hour'] = row['hour'].replace("A","AND")
    row['week_day'] = row['week_day'].replace("LUNES","Monday")
    row['week_day'] = row['week_day'].replace("MARTES","Tuesday")
    row['week_day'] = row['week_day'].replace("MIERCOLES","Wednesday")
    row['week_day'] = row['week_day'].replace("JUEVES","Thursday")
    row['week_day'] = row['week_day'].replace("VIERNES","Friday")
    row['week_day'] = row['week_day'].replace("SABADO","Saturday")
    row['week_day'] = row['week_day'].replace("DOMINGO","Sunday")
    #I'm going to transform all this columns to numerical columns with a 0 if the answer is No and 1 if it is yes, and then i can sum the yes in tableau and obtain the different quantity of causes. 
    row['ac_hail'] = row['ac_hail'].replace("SI","1")
    row['ac_ice'] = row['ac_ice'].replace("SI","1")
    row['ac_rain'] = row['ac_rain'].replace("SI","1")
    row['ac_fog'] = row['ac_fog'].replace("SI","1")
    row['ac_dry'] = row['ac_dry'].replace("SI","1")
    row['ac_snow'] = row['ac_snow'].replace("SI","1")
    row['wet'] = row['wet'].replace("SI","1")
    row['oil'] = row['oil'].replace("SI","1")
    row['mud'] = row['mud'].replace("SI","1")
    row['gravel'] = row['gravel'].replace("SI","1")
    row['ice'] = row['ice'].replace("SI","1")
    row['dry_and_clean'] = row['dry_and_clean'].replace("SI","1")
    row['ac_hail'] = row['ac_hail'].replace("NO","0")
    row['ac_ice'] = row['ac_ice'].replace("NO","0")
    row['ac_rain'] = row['ac_rain'].replace("NO","0")
    row['ac_fog'] = row['ac_fog'].replace("NO","0")
    row['ac_dry'] = row['ac_dry'].replace("NO","0")
    row['ac_snow'] = row['ac_snow'].replace("NO","0")
    row['wet'] = row['wet'].replace("NO","0")
    row['oil'] = row['oil'].replace("NO","0")
    row['mud'] = row['mud'].replace("NO","0")
    row['gravel'] = row['gravel'].replace("NO","0")
    row['ice'] = row['ice'].replace("NO","0")
    row['dry_and_clean'] = row['dry_and_clean'].replace("NO","0")
    row['ac_hail'] = pd.to_numeric(row['ac_hail'])
    row['ac_ice'] = pd.to_numeric(row['ac_ice'])
    row['ac_rain'] = pd.to_numeric(row['ac_rain'])
    row['ac_fog'] = pd.to_numeric(row['ac_fog'])
    row['ac_dry'] = pd.to_numeric(row['ac_dry'])
    row['ac_snow'] = pd.to_numeric(row['ac_snow'])
    row['wet'] = pd.to_numeric(row['wet'])
    row['oil'] = pd.to_numeric(row['oil'])
    row['mud'] = pd.to_numeric(row['mud'])
    row['gravel'] = pd.to_numeric(row['gravel'])
    row['ice'] = pd.to_numeric(row['ice'])
    row['dry_and_clean'] = pd.to_numeric(row['dry_and_clean'])
    row['type_of_accident'] = row['type_of_accident'].replace("COLISIÓN DOBLE","multiple crash")
    row['type_of_accident'] = row['type_of_accident'].replace("ATROPELLO","run over")
    row['type_of_accident'] = row['type_of_accident'].replace("COLISIÓN MÚLTIPLE","multiple crash")
    row['type_of_accident'] = row['type_of_accident'].replace("CHOQUE CON OBJETO FIJO","crash with a fixed object")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA MOTOCICLETA","motorcycle crash")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA VIAJERO BUS","other")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA BICICLETA","other")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA CICLOMOTOR","other")
    row['type_of_accident'] = row['type_of_accident'].replace("OTRAS CAUSAS","other")
    row['type_of_accident'] = row['type_of_accident'].replace("VUELCO","other")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA VEHÍCULO 3 RUEDAS","other")
    row['vehicle'] = row['vehicle'].replace("TURISMO","car")
    row['vehicle'] = row['vehicle'].replace("MOTOCICLETA","motorcycle")
    row['vehicle'] = row['vehicle'].replace("NO ASIGNADO","unknown")
    row['vehicle'] = row['vehicle'].replace("FURGONETA","van")
    row['vehicle'] = row['vehicle'].replace("AUTOBUS-AUTOCAR","bus")
    row['vehicle'] = row['vehicle'].replace("AUTO-TAXI","car")
    row['vehicle'] = row['vehicle'].replace("BICICLETA","bycicle")
    row['vehicle'] = row['vehicle'].replace("CICLOMOTOR","other")
    row['vehicle'] = row['vehicle'].replace("CAMION","other")
    row['vehicle'] = row['vehicle'].replace("VARIOS","other")
    row['vehicle'] = row['vehicle'].replace("AMBULANCIA","other")
    row['vehicle'] = row['vehicle'].replace("VEH.3 RUEDAS","other")
    row['type_of_person'] = row['type_of_person'].replace("PEATON","pedestrian")
    row['type_of_person'] = row['type_of_person'].replace("CONDUCTOR","driver")
    row['type_of_person'] = row['type_of_person'].replace("VIAJERO","traveler")
    row['type_of_person'] = row['type_of_person'].replace("PEATON","pedestrian")
    row['gender'] = row['gender'].replace("HOMBRE","male")
    row['gender'] = row['gender'].replace("MUJER","female")
    row['gender'] = row['gender'].replace("NO ASIGNADO","unknown")
    row['gravity'] = row['gravity'].replace("HL","mild injury")
    row['gravity'] = row['gravity'].replace("IL","uninjured")
    row['gravity'] = row['gravity'].replace("NO ASIGNADA","unknown")
    row['gravity'] = row['gravity'].replace("HG","serious injury")
    row['gravity'] = row['gravity'].replace("MT","death")
    row['age'] = row['age'].replace("DESCONOCIDA","unknown")
    pattern2 = 'MAS '
    pattern3 = 'A[NÑ]OS'
    for i in range(len(row['hour'])):
        if row['age'][i] == 'unknown':
            row['age'][i] = row['age'][i]
        else:
            row['age'][i] = re.sub(pattern2, '+',row['age'][i])
            row['age'][i] = re.sub(pattern, '',row['age'][i])
            row['age'][i] = re.sub(pattern1, '-',row['age'][i])
            row['age'][i] = re.sub(pattern3, 'years old',row['age'][i])
    #row['age'] = row['age'].replace("DE","BETWEEN")
    #row['age'] = row['age'].replace("A","AND")
    #row['age'] = row['age'].replace("AÑOS","years old")
    return row

In [29]:
clean_accidents = translate(total_accidents)
clean_accidents

<ipython-input-28-a3ecc0faabb8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['hour'][i] = re.sub(pattern, '',row['hour'][i])
<ipython-input-28-a3ecc0faabb8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['hour'][i] = re.sub(pattern1, '-',row['hour'][i])
<ipython-input-28-a3ecc0faabb8>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['age'][i] = re.sub(pattern2, '+',row['age'][i])
<ipython-input-28-a3ecc0faabb8>:97: S

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,gravel,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,male,mild injury,6-9 years old
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,male,mild injury,40-44 years old
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,male,mild injury,35-39 years old
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,female,mild injury,35-39 years old
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,car,driver,male,uninjured,30-34 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,0,1,1,multiple crash,car,traveler,male,uninjured,30-34 years old
78032,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,0,1,1,multiple crash,car,driver,male,uninjured,40-44 years old
78033,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,0,1,1,other,car,driver,female,uninjured,21-24 years old
78034,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,0,1,1,other,car,driver,male,mild injury,50-54 years old


In [30]:
for col in clean_accidents.columns:
    print(clean_accidents[col].unique())

['2016-01-01T00:00:00.000000000' '2016-01-02T00:00:00.000000000'
 '2016-01-03T00:00:00.000000000' ... '2018-12-29T00:00:00.000000000'
 '2018-12-30T00:00:00.000000000' '2018-12-31T00:00:00.000000000']
['00:00-00:59' '1:00-1:59' '2:00-2:59' '3:00-3:59' '5:00-5:59' '6:00-6:59'
 '9:00-9:59' '11:00-11:59' '12:00-12:59' '14:00-14:59' '16:00-16:59'
 '18:00-18:59' '19:00-19:59' '20:00-20:59' '21:00-21:59' '22:00-22:59'
 '23:00-23:59' '15:00-15:59' '17:00-17:59' '10:00-10:59' '13:00-13:59'
 '7:00-7:59' '8:00-8:59' '4:00-4:59']
['Friday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday']
['PUENTE DE VALLECAS' 'CIUDAD LINEAL' 'CHAMARTIN' 'USERA' 'LATINA'
 'RETIRO' 'VILLAVERDE' 'SALAMANCA' 'CHAMBERI' 'ARGANZUELA' 'TETUAN'
 'FUENCARRAL-EL PARDO' 'HORTALEZA' 'CENTRO' 'BARAJAS' 'CARABANCHEL'
 'VILLA DE VALLECAS' 'SAN BLAS' 'MONCLOA-ARAVACA' 'VICALVARO' 'MORATALAZ']
['AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE DARIEN'
 'AVENIDA DEL MARQUES DE CORBERA NUM' 'AUTOVIA  M-30 CALZADA 1 KM.' ..

In [31]:
clean_accidents.isna().sum()

date                 0
hour                 0
week_day             0
district             0
street               0
number               0
id_accident          0
ac_hail              0
ac_ice               0
ac_rain              0
ac_fog               0
ac_dry               0
ac_snow              0
wet                  0
oil                  0
mud                  0
gravel               0
ice                  0
dry_and_clean        0
number_of_victims    0
type_of_accident     0
vehicle              0
type_of_person       0
gender               0
gravity              0
age                  0
dtype: int64

In [ ]:
# I'm going to check if all unknown vehicles are because type of people are pedestrians

In [32]:
clean_accidents['vehicle'].value_counts()

car           47977
motorcycle    12865
unknown        5208
other          3674
van            3476
bus            2624
bycicle        2212
Name: vehicle, dtype: int64

In [33]:
clean_accidents['type_of_person'].value_counts()

driver        57013
traveler      15858
pedestrian     5165
Name: type_of_person, dtype: int64

In [ ]:
# Not all, but probably most of them

In [34]:
for i in range(len(clean_accidents['vehicle'])):
    if (clean_accidents['vehicle'][i] == 'unknown') & (clean_accidents['type_of_person'][i] == 'pedestrian'):
        clean_accidents['vehicle'][i] = 'none'

<ipython-input-34-6e9585f11168>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['vehicle'][i] = 'none'


In [35]:
clean_accidents['vehicle'].value_counts()

car           47977
motorcycle    12865
none           5159
other          3674
van            3476
bus            2624
bycicle        2212
unknown          49
Name: vehicle, dtype: int64

In [36]:
pedestrian = clean_accidents[clean_accidents['type_of_person'] =='pedestrian']
pedestrian['vehicle'].value_counts()

none     5159
other       3
car         2
van         1
Name: vehicle, dtype: int64

In [ ]:
# If they are pedestrians, it is impossible they were on a car or a van, so i'm going to change it. Other it's possible because they can have an skate or something like that.

In [37]:
for i in range(len(clean_accidents['vehicle'])):
    if ((clean_accidents['vehicle'][i] == 'car') or (clean_accidents['vehicle'][i] == 'van')) & (clean_accidents['type_of_person'][i] == 'pedestrian'):
        clean_accidents['vehicle'][i] = 'none'

<ipython-input-37-5d4695f3e438>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['vehicle'][i] = 'none'


In [38]:
pedestrian = clean_accidents[clean_accidents['type_of_person'] =='pedestrian']
pedestrian['vehicle'].value_counts()

none     5162
other       3
Name: vehicle, dtype: int64

In [41]:
codes = pd.read_excel('./files_for_lab/codes.xlsx')
codes

,DISTRICT,CODE
0,USERA,12
1,SAN BLAS,20
2,FUENCARRAL-EL PARDO,8
3,ARGANZUELA,2
4,SALAMANCA,4
5,CARABANCHEL,11
6,TETUAN,6
7,CHAMBERI,7
8,VILLAVERDE,17
9,PUENTE DE VALLECAS,13


In [42]:
#I'm going to add a new column with the district code

In [43]:
clean_accidents['district_code'] = 0
for r in range(len(codes['DISTRICT'])):
    for i in range(len(clean_accidents['district'])):
        if clean_accidents['district'][i] == codes['DISTRICT'][r]:
            clean_accidents['district_code'][i] += codes['CODE'][r]

<ipython-input-43-c437499abf76>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['district_code'][i] += codes['CODE'][r]


In [44]:
clean_accidents

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,40-44 years old,13
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,35-39 years old,13
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,female,mild injury,35-39 years old,13
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,car,driver,male,uninjured,30-34 years old,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,1,1,multiple crash,car,traveler,male,uninjured,30-34 years old,4
78032,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,40-44 years old,4
78033,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,1,1,other,car,driver,female,uninjured,21-24 years old,16
78034,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,1,1,other,car,driver,male,mild injury,50-54 years old,16


In [115]:
max(clean_accidents['district_code'])

21

In [46]:
accidents = clean_accidents[['date', 'hour' , 'week_day', 'district', 'street','id_accident', 'number_of_victims', 'type_of_accident', 'vehicle', 'type_of_person',
       'gender', 'gravity', 'age', 'district_code']]

In [47]:
accidents.to_excel('./files_for_lab/accidents.xlsx')
accidents.to_csv('./files_for_lab/accidents.csv')

In [ ]:
# I'm going to use this file to obtain postal codes in MYSQL using the postal_codes file.

In [48]:
postal_codes = pd.read_csv('./files_for_lab/postal_codes.csv', sep=';', engine='python')
postal_codes

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Tipo de tramo,Nombre de la aproximacion,Numero inicial del tramo,Calificador del numero inicial del tramo,Numero final del tramo,Calificador del numero final del tramo,Codigo postal,Seccion de carteria
0,127,CALLE,DE,ISABEL COLBRAND,par,NUM,2,,12,,28050,N/D
1,127,CALLE,DE,ISABEL COLBRAND,par,NUM,14,,22,,28050,N/D
2,127,CALLE,DE,ISABEL COLBRAND,par,NUM,24,,9998,,28050,N/D
3,150,CALLE,DE,GONZALEZ DAVILA,impar,NUM,1,,9999,,28031,N/D
4,150,CALLE,DE,GONZALEZ DAVILA,par,NUM,2,,9998,,28031,N/D
...,...,...,...,...,...,...,...,...,...,...,...,...
22916,31004755,TRAVESIA,DE,CAPRI,par,NUM,2,,9998,,28032,N/D
22917,31004759,CALLE,DE,ANGELES GARCIA MADRID,impar,NUM,1,,9999,,28007,N/D
22918,31004759,CALLE,DE,ANGELES GARCIA MADRID,par,NUM,2,,9998,,28007,N/D
22919,31004764,PLAZA,DE LOS,CHISPEROS DE CHAMBERI,impar,NUM,1,,9999,,28010,N/D


In [ ]:
# I'm not going to translate this table because it's not important. I'm going to use this table and street_codes to obtain the postal code.

In [49]:
postal_codes['Codigo postal'].nunique()

56

In [52]:
postal_code_finished = postal_codes[['Codigo de via', 'Clase de la via','Particula de la via', 'Nombre de la via', 'Codigo postal']]
postal_code_finished

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Codigo postal
0,127,CALLE,DE,ISABEL COLBRAND,28050
1,127,CALLE,DE,ISABEL COLBRAND,28050
2,127,CALLE,DE,ISABEL COLBRAND,28050
3,150,CALLE,DE,GONZALEZ DAVILA,28031
4,150,CALLE,DE,GONZALEZ DAVILA,28031
...,...,...,...,...,...
22916,31004755,TRAVESIA,DE,CAPRI,28032
22917,31004759,CALLE,DE,ANGELES GARCIA MADRID,28007
22918,31004759,CALLE,DE,ANGELES GARCIA MADRID,28007
22919,31004764,PLAZA,DE LOS,CHISPEROS DE CHAMBERI,28010


In [61]:
code_postal_code = pd.read_excel('./files_for_lab/code_postal_code.xlsx')
code_postal_code

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Cantidad de numeraciones en la via,Distritos atravesados,Codigos postales,Campo que indica si tiene zonas en S E R,Tipo de la via,Situacion de la via respecto al terreno,Tipo de denominación de la vía,Superficie aproximada m2 (aceras-isletas),Superficie aproximada m2 (calzada),Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,52,8,28050,NO,Vía,Nivel,Pleno,11858,16916,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18,18,varios,NO,Vía,Nivel,Pleno,6655,13875,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,12,2021-01-01 00:00:00,varios,SI,Vía,Nivel,Pleno,2145,0,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,19,1,varios,SI,Vía,Nivel,Pleno,897,0,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,7,6,varios,SI,Vía,Nivel,Pleno,131,314,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9846,CAÑADA,,REAL SECTOR 5,31005678,140,19,,NO,Topónimo,Nivel,Admon,0,0,CAÑADA REAL SECTOR 5 ...
9847,CAÑADA,,REAL SECTOR 3,31005679,48,19,,NO,Topónimo,Nivel,Admon,0,0,CAÑADA REAL SECTOR 3 ...
9848,CAÑADA,,REAL SECTOR 4,31005680,89,19,,NO,Topónimo,Nivel,Pleno,0,0,CAÑADA REAL SECTOR 4 ...
9849,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,10,2,,SI,Vía,Nivel,Pleno,0,0,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [63]:
code_postal_code.columns

Index(['Clase de la via', 'Particula de la via', 'Nombre de la via',
       'Codigo de vía', 'Cantidad de numeraciones en la via',
       'Distritos atravesados', 'Codigos postales',
       'Campo que indica si tiene zonas en S E R ', 'Tipo de la via',
       'Situacion de la via respecto al terreno',
       'Tipo de denominación de la vía ',
       'Superficie aproximada m2 (aceras-isletas)',
       'Superficie aproximada m2 (calzada)', 'Literal completo del vial '],
      dtype='object')

In [93]:
code_postal_code_finished = code_postal_code[['Clase de la via', 'Particula de la via', 'Nombre de la via', 'Codigo de vía', 'Distritos atravesados', 'Literal completo del vial ']]
code_postal_code_finished

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Distritos atravesados,Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,8,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,2021-01-01 00:00:00,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,1,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,6,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...
9846,CAÑADA,,REAL SECTOR 5,31005678,19,CAÑADA REAL SECTOR 5 ...
9847,CAÑADA,,REAL SECTOR 3,31005679,19,CAÑADA REAL SECTOR 3 ...
9848,CAÑADA,,REAL SECTOR 4,31005680,19,CAÑADA REAL SECTOR 4 ...
9849,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,2,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [66]:
code_postal_code_finished.dtypes

Clase de la via               object
Particula de la via           object
Nombre de la via              object
Codigo de vía                  int64
Distritos atravesados         object
Literal completo del vial     object
dtype: object

In [68]:
code_postal_code_finished['Distritos atravesados'].unique()

array([8, 18, datetime.datetime(2021, 1, 1, 0, 0), 1, 6, 15, 20,
       datetime.datetime(2021, 7, 4, 0, 0), 16, 3, 5, 2, 9, 14, 21,
       datetime.datetime(2021, 9, 8, 0, 0), 13,
       '16-20                                                       ',
       '08-16-21                                                    ', 17,
       '16-21                                                       ', 12,
       7, 10, datetime.datetime(2021, 8, 4, 0, 0),
       '14-15                                                       ',
       '08-15-16-20-21                                              ',
       datetime.datetime(2017, 12, 1, 0, 0), 11,
       datetime.datetime(2018, 8, 1, 0, 0),
       datetime.datetime(2018, 9, 1, 0, 0),
       datetime.datetime(2016, 8, 1, 0, 0),
       datetime.datetime(2021, 12, 11, 0, 0),
       '                                                            ',
       datetime.datetime(2021, 8, 1, 0, 0),
       datetime.datetime(2021, 8, 7, 0, 0),
       datetime.dat

In [ ]:
# This column is one of the most important because it's a link between the two tables i need in mysql to obtain the postal code. I need to clean it.
# Datetime is an error, the day or the month are the different districts. Day is the main district and month a district where the street pass but not a lot. In day it's the most part of the street.
# And there are a lot of streets with many code districts, so i'm going to use random to pick one because i need unique values.

In [70]:
import random
import time
import datetime
from datetime import date

In [94]:
for i in range(len(code_postal_code_finished['Distritos atravesados'])):
    if isinstance( code_postal_code_finished['Distritos atravesados'][i], str ) == True:
        code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i].strip()

<ipython-input-94-10452abc5287>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i].strip()
C:\Users\Propietario\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [95]:
for i in range(len(code_postal_code_finished['Distritos atravesados'])):
    if code_postal_code_finished['Distritos atravesados'][i] == '':
        code_postal_code_finished['Distritos atravesados'] = code_postal_code_finished['Distritos atravesados']
    else:
        if (isinstance( code_postal_code_finished['Distritos atravesados'][i], str ) == False) & (isinstance( code_postal_code_finished['Distritos atravesados'][i], int ) == False):
            day = code_postal_code_finished['Distritos atravesados'][i].day
            code_postal_code_finished['Distritos atravesados'][i] = day
        else:
            if isinstance( code_postal_code_finished['Distritos atravesados'][i], int ) == True:
                code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i]
            else:
                pattern = '\d+'
                list_ = re.findall(pattern, code_postal_code_finished['Distritos atravesados'][i])
                winner = random.choice(list_)
                code_postal_code_finished['Distritos atravesados'][i] = winner
code_postal_code_finished['Distritos atravesados'] = pd.to_numeric(code_postal_code_finished['Distritos atravesados'])

<ipython-input-95-77a26863843f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i]
<ipython-input-95-77a26863843f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Distritos atravesados'][i] = day
<ipython-input-95-77a26863843f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finishe

In [96]:
code_postal_code_finished

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Distritos atravesados,Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,8.0,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18.0,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,1.0,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,1.0,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,6.0,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...
9846,CAÑADA,,REAL SECTOR 5,31005678,19.0,CAÑADA REAL SECTOR 5 ...
9847,CAÑADA,,REAL SECTOR 3,31005679,19.0,CAÑADA REAL SECTOR 3 ...
9848,CAÑADA,,REAL SECTOR 4,31005680,19.0,CAÑADA REAL SECTOR 4 ...
9849,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,2.0,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [100]:
code_postal_code_finished['Distritos atravesados'].unique()

array([ 8., 18.,  1.,  6., 15., 20.,  4., 16.,  3.,  5.,  2.,  9., 14.,
       21., 13., 17., 12.,  7., 10., 11., nan, 19.])

In [ ]:
# The NAN aare the empty spaces, so i can drop all these rows

In [102]:
code_postal_code_finished['Distritos atravesados'].value_counts(dropna = False)

8.0     850
9.0     825
11.0    746
1.0     731
16.0    670
13.0    635
20.0    566
10.0    512
5.0     500
15.0    491
6.0     428
17.0    425
18.0    386
12.0    373
21.0    305
19.0    291
2.0     269
4.0     266
3.0     261
7.0     204
14.0    115
NaN       2
Name: Distritos atravesados, dtype: int64

In [110]:
code_postal_code_finished = code_postal_code_finished.dropna()
code_postal_code_finished = code_postal_code_finished.reset_index(drop=True)
code_postal_code_finished

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Distritos atravesados,Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,8.0,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18.0,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,1.0,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,1.0,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,6.0,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...
9844,CAÑADA,,REAL SECTOR 5,31005678,19.0,CAÑADA REAL SECTOR 5 ...
9845,CAÑADA,,REAL SECTOR 3,31005679,19.0,CAÑADA REAL SECTOR 3 ...
9846,CAÑADA,,REAL SECTOR 4,31005680,19.0,CAÑADA REAL SECTOR 4 ...
9847,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,2.0,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [105]:
postal_code_finished['Codigo postal'].value_counts(dropna=False)

28023    811
28011    801
28021    793
28042    722
28043    677
28019    675
28053    655
28039    628
28033    625
28025    624
28022    624
28035    613
28002    604
28041    574
28026    560
28037    560
28027    550
28031    547
28017    526
28034    517
28018    499
28005    498
28038    474
28029    451
28032    433
28044    426
28028    424
28045    417
28050    395
28007    335
28012    321
28055    321
28004    318
28016    308
28051    293
28054    292
28030    288
28049    287
28047    278
28013    267
28020    266
28024    243
28052    235
28015    219
28010    218
28036    211
28003    207
28008    207
28014    199
28009    191
28006    177
28040    176
28048    151
28001    134
28046     72
28760      4
Name: Codigo postal, dtype: int64

In [106]:
def deleting_commas(row):
    for i in range(len(row['Nombre de la via'])):
        row['Nombre de la via'][i] = row['Nombre de la via'][i].replace(",","-")
    return row

In [107]:
postal_code_finished = deleting_commas(postal_code_finished)

<ipython-input-106-cc3ee0bed7a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Nombre de la via'][i] = row['Nombre de la via'][i].replace(",","-")
<ipython-input-107-7aa09d19ab1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postal_code_finished = deleting_commas(postal_code_finished)


In [111]:
code_postal_code_finished = deleting_commas(code_postal_code_finished)

<ipython-input-106-cc3ee0bed7a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Nombre de la via'][i] = row['Nombre de la via'][i].replace(",","-")


In [112]:
for i in range(len(code_postal_code_finished['Literal completo del vial '])):
    code_postal_code_finished['Literal completo del vial '][i] = code_postal_code_finished['Literal completo del vial '][i].replace(",","-")

<ipython-input-112-df1b564ee1d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Literal completo del vial '][i] = code_postal_code_finished['Literal completo del vial '][i].replace(",","-")


In [116]:
postal_code_finished = postal_code_finished.drop_duplicates()

In [117]:
postal_code_finished = postal_code_finished.reset_index(drop=True)

In [118]:
postal_code_finished

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Codigo postal
0,127,CALLE,DE,ISABEL COLBRAND,28050
1,150,CALLE,DE,GONZALEZ DAVILA,28031
2,200,CALLE,DE LA,ABADA,28013
3,300,CALLE,DE LOS,ABADES,28012
4,400,CALLE,DE LA,ABADESA,28039
...,...,...,...,...,...
9249,31004752,TRAVESIA,DE LA,ISLA DE JAVA,28034
9250,31004754,TRAVESIA,DE,AQUITANIA,28032
9251,31004755,TRAVESIA,DE,CAPRI,28032
9252,31004759,CALLE,DE,ANGELES GARCIA MADRID,28007


In [119]:
postal_code_finished.to_csv('./files_for_lab/postal_code_finished.csv')
code_postal_code_finished.to_csv('./files_for_lab/code_postal_code_finished.csv')

In [50]:
password = getpass.getpass()

 ········


In [ ]:
# READING THE DATA
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)
data = pd.read_sql('''SELECT i.inventory_id, f.film_id, f.title, f.rental_duration, f.rental_rate, f.length, f.replacement_cost,  f.rating, f.special_features, i.store_id,  r.staff_id, r.rental_date,
CASE
WHEN r.rental_date BETWEEN '2005-01-01' AND '2005-08-1' THEN r.rental_date
ELSE NULL
END AS last_time_rented,
CASE
WHEN MAX(r.rental_date) BETWEEN '2005-08-01' AND '2005-08-31' THEN 1
ELSE 0
END AS rented_last_month, COUNT(r.inventory_id) AS times_rented,
CASE
WHEN MIN(r.rental_date) BETWEEN '2005-01-01' AND '2005-08-1' THEN 1
ELSE 0
END AS rented_before_august
FROM sakila.film f
LEFT JOIN sakila.inventory i USING (film_id)
LEFT JOIN sakila.rental r USING (inventory_id)
WHERE (r.rental_date BETWEEN '2005-01-01' AND '2005-08-31') OR r.rental_date IS NULL
GROUP BY i.inventory_id
ORDER BY i.inventory_id ASC;''', engine)

In [ ]:
data